<a href="https://colab.research.google.com/github/gomlfx/apiaryfund/blob/master/random_forest_regression_2_eurusd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
#using eurusd, change to your preferred asset
eu = pd.read_csv('EURUSD1440.csv')
eu.columns=['Date','Time','eu_Open','eu_High','eu_Low','eu_Close','Volume']
#Use D1 at least 2000 observations, change below for the number of rows you have.
#session crash or slow computing time may require you to use less rows. 
eu = eu.iloc[-2000:] 
#eu = eu.iloc[-8000:] 
eu = eu.reset_index(drop=True)
eu['Index']=eu.index
eu.set_index('Index', drop=True, append=False, inplace=True, verify_integrity=False)
#and make index a feature
eu.reset_index(inplace=True)
#drop last rows if needed
eu = eu[:-1]
print(eu.head())
print(eu.tail())

   Index        Date   Time  eu_Open  eu_High   eu_Low  eu_Close  Volume
0      0  2014.02.14  00:00  1.36785  1.37145  1.36735   1.36918   33267
1      1  2014.02.17  00:00  1.37007  1.37240  1.36919   1.37047   24544
2      2  2014.02.18  00:00  1.37047  1.37696  1.36943   1.37604   32235
3      3  2014.02.19  00:00  1.37601  1.37730  1.37242   1.37321   28863
4      4  2014.02.20  00:00  1.37322  1.37623  1.36857   1.37202   34443
      Index        Date   Time  eu_Open  eu_High   eu_Low  eu_Close  Volume
1994   1994  2021.10.25  00:00  1.16398  1.16650  1.15906   1.16100   72911
1995   1995  2021.10.26  00:00  1.16083  1.16256  1.15848   1.15956   68126
1996   1996  2021.10.27  00:00  1.15868  1.16257  1.15848   1.16025   85984
1997   1997  2021.10.28  00:00  1.16018  1.16919  1.15821   1.16822  112541
1998   1998  2021.10.29  00:00  1.16794  1.16899  1.15350   1.15622  100657


In [2]:
#for close prediction
#select rows to use for x,y
x = eu.iloc[:,[0]].values
print(x)
y = eu.iloc[:,[6]].values
print(y)

[[   0]
 [   1]
 [   2]
 ...
 [1996]
 [1997]
 [1998]]
[[1.36918]
 [1.37047]
 [1.37604]
 ...
 [1.16025]
 [1.16822]
 [1.15622]]


In [3]:
#for close prediction
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 10000, random_state = 0)

# fit the regressor with x and y data
regressor.fit(x, y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [4]:
#for close prediction
Y_pred = regressor.predict(np.array([1999]).reshape(1, 1)) # test the output by changing values
Y_pred

array([1.15966956])

In [5]:
# #for close prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), .01)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.plot(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(80,100)
# plt.ylim(1.15,1.167)
# plt.show()


In [6]:
#for low prediction
#select rows to use for x,y
x = eu.iloc[:,[0]].values
print(x)
y = eu.iloc[:,[5]].values
print(y)

[[   0]
 [   1]
 [   2]
 ...
 [1996]
 [1997]
 [1998]]
[[1.36735]
 [1.36919]
 [1.36943]
 ...
 [1.15848]
 [1.15821]
 [1.1535 ]]


In [7]:
#for low prediction
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 10000, random_state = 0)

# fit the regressor with x and y data
regressor.fit(x, y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [8]:
#for low prediction
Y_pred = regressor.predict(np.array([1999]).reshape(1, 1)) # test the output by changing values
Y_pred

array([1.15539656])

In [9]:
# #for low prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), .01)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.plot(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(80,100)
# plt.ylim(1.15,1.167)
# plt.show()

In [10]:
#for high prediction
#select rows to use for x,y
x = eu.iloc[:,[0]].values
print(x)
y = eu.iloc[:,[4]].values
print(y)

[[   0]
 [   1]
 [   2]
 ...
 [1996]
 [1997]
 [1998]]
[[1.37145]
 [1.3724 ]
 [1.37696]
 ...
 [1.16257]
 [1.16919]
 [1.16899]]


In [11]:
#for high prediction
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 10000, random_state = 0)

# fit the regressor with x and y data
regressor.fit(x, y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [12]:
#for high prediction
Y_pred = regressor.predict(np.array([1999]).reshape(1, 1)) # test the output by changing values
Y_pred

array([1.16828097])

In [13]:
# #for high prediction
# # Visualising the Random Forest Regression results

# # arange for creating a range of values
# # from min value of x to max
# # value of x with a difference of 0.01
# # between two consecutive values
# X_grid = np.arange(min(x), max(x), .01)

# # reshape for reshaping the data into a len(X_grid)*1 array,
# # i.e. to make a column out of the X_grid value				
# X_grid = X_grid.reshape((len(X_grid), 1))

# # Scatter plot for original data
# plt.scatter(x, y, color = 'blue')

# # plot predicted data
# plt.plot(X_grid, regressor.predict(X_grid),
# 		color = 'green')
# plt.title('Random Forest Regression')
# plt.xlabel('Position level')
# plt.ylabel('EURUSD')
# plt.xlim(80,100)
# plt.ylim(1.15,1.167)
# plt.show()